- http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
- http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
- http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Обогащение данных

In [2]:
data = pd.read_csv('energy_0.csv')
print(data.head())

   Unnamed: 0            timestamp  site_id  building_id  meter  \
0           0  2016-01-01 00:00:00        0            0      0   
1           1  2016-01-01 01:00:00        0            0      0   
2           2  2016-01-01 02:00:00        0            0      0   
3           3  2016-01-01 03:00:00        0            0      0   
4           4  2016-01-01 04:00:00        0            0      0   

   meter_reading primary_use  square_feet  year_built  floor_count  \
0            0.0   Education         7432      2008.0          NaN   
1            0.0   Education         7432      2008.0          NaN   
2            0.0   Education         7432      2008.0          NaN   
3            0.0   Education         7432      2008.0          NaN   
4            0.0   Education         7432      2008.0          NaN   

   air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
0             25.0             6.0             20.0                0.0   
1             24.4          

In [3]:
data['wind_direction_rad'] = data['wind_direction']*np.pi/180
data['wind_direction_sin'] = np.sin(data['wind_direction_rad'])
data['wind_direction_cos'] = np.cos(data['wind_direction_rad'])
data = data.drop(labels=['wind_direction_rad'], axis=1)

Производная температуры

In [4]:
data['air_temperature1'] = data['air_temperature'].diff()
data.at[0, 'air_temperature1'] = data.at[1, 'air_temperature1']

удаляем бесполезные данные


In [5]:
data = data.loc[data['meter_reading'] > 0]

Удаляем столбцы

In [6]:
data = data.drop(labels = ['site_id','building_id', 'meter', 'primary_use', 'square_feet', 
                           'year_built', 'floor_count', 'wind_direction'], axis=1)
print(data.head())

      Unnamed: 0            timestamp  meter_reading  air_temperature  \
704          704  2016-01-30 08:00:00        43.6839              8.3   
725          725  2016-01-31 05:00:00        37.5408             12.8   
737          737  2016-01-31 17:00:00        52.5571             20.6   
2366        2366  2016-04-08 14:00:00        59.3827             21.7   
2923        2923  2016-05-01 19:00:00       448.0000             31.1   

      cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
704              0.0              6.1                0.0              1019.0   
725              0.0             10.0                0.0              1021.9   
737              0.0             11.7                0.0              1020.9   
2366             2.0             14.4                0.0              1015.1   
2923             0.0             17.2                0.0              1016.1   

      wind_speed  wind_direction_sin  wind_direction_cos  air_temperature1  
704

In [7]:
data.to_csv('energy_1.csv', index=False)

## Разделение данных
на обучающую и проверочную выборки 80/20

In [8]:
data_train, data_test = train_test_split(data, test_size=0.2)
print(data_train.head())

      Unnamed: 0            timestamp  meter_reading  air_temperature  \
7106        7106  2016-10-23 02:00:00       167.2270             16.7   
6591        6591  2016-10-01 15:00:00       236.8480             28.3   
8312        8312  2016-12-12 08:00:00        81.9072             20.6   
7693        7693  2016-11-16 13:00:00       183.6090             15.0   
7572        7572  2016-11-11 12:00:00       171.3230             12.8   

      cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
7106             0.0              8.9                0.0              1018.4   
6591             4.0             22.2                0.0              1017.6   
8312             0.0             19.4                0.0              1020.0   
7693             0.0             11.1                0.0              1015.2   
7572             4.0             11.7                0.0              1019.5   

      wind_speed  wind_direction_sin  wind_direction_cos  air_temperature1  
710

## Построение модели по обучающей выборке
среднее занчение

In [9]:
model = float(data_train["meter_reading"].mean())
print('{0:.4}'.format(model))

237.4


## Проверка модели
на проверочной выборке

In [11]:
err = np.mean(np.abs(data_test['meter_reading'] - model))
print('{0:.4}%'.format(100*err/model))

15.28%


## Задание
1. Выделить час из timestamp
2. Вычислить среднее энергопотребления по часам на обучающей выборке
3. Проверить точность работы модели "среднее по часам" относительно модели "общее среднее" напроверочной выборке